In [78]:
import sys
sys.path.append('../src')
import numpy as np
from numpy.random import normal, multivariate_normal
import P_binary
import P_random
from astropy.table import Table
import pickle
import time

In [2]:
print "Opening Tycho-2 catalog..."
# Read in sample from Tycho-2 table
filename = ('../data/tycho-2/tyc2.dat')
readme = ('../data/tycho-2/ReadMe')
tycho_full = Table.read(filename, format='cds', guess=False, readme=readme)
print "...finished reading data."


# Create the clean tycho-2 catalog
dtype = [('ID','i8'),('ra','f8'),('dec','f8'),('mu_ra','f8'),('mu_dec','f8'), \
         ('mu_ra_err','f8'),('mu_dec_err','f8'),('Bmag','f8'),('Vmag','f8')]

ids = np.intersect1d(np.where(tycho_full['q_pmRA'] >= 0.1), np.where(tycho_full['q_pmDE'] >= 0.1))

t = np.zeros(len(ids), dtype=dtype)
t['ID'] = tycho_full['TYC1'][ids]*100000 + tycho_full['TYC2'][ids]
t['ra'] = tycho_full['RAmdeg'][ids]
t['dec'] = tycho_full['DEmdeg'][ids]
t['mu_ra'] = tycho_full['pmRA'][ids]
t['mu_dec'] = tycho_full['pmDE'][ids]
t['mu_ra_err'] = tycho_full['e_pmRA'][ids]
t['mu_dec_err'] = tycho_full['e_pmDE'][ids]
t['Bmag'] = tycho_full['BTmag'][ids]
t['Vmag'] = tycho_full['VTmag'][ids]


Opening Tycho-2 catalog...
...finished reading data.


In [3]:
# Generate simulated binaries
print "Generating binaries..."
P_binary.generate_binary_set(num_sys=100000)

Generating binaries...


In [4]:
# Generate random alignment KDEs using first entry as a test position
pos_density = P_random.get_sigma_pos(t['ra'][0], t['dec'][0], catalog=t, method='kde')
pm_density = P_random.get_sigma_mu(t['mu_ra'][0], t['mu_dec'][0], catalog=t, method='kde')

In [5]:
def get_ang_sep(ra1, dec1, ra2, dec2):
    rad_dec1 = np.pi/180.0 * dec1
    rad_dec2 = np.pi/180.0 * dec2
    return np.sqrt( (ra1-ra2)**2 * np.cos(rad_dec1) * np.cos(rad_dec2) + (dec1-dec2)**2 )

In [83]:
size_integrate = 10  # Number of samples for delta mu integration
size_integrate_full = 10000
f_bin = 0.5  # binary fraction

start = time.time()

# Now, let's calculate the probabilities
length = len(t)
print "We are testing", length, "stars..."

dtype = [('i_1','i4'),('i_2','i4'),('ID_1','i4'),('ID_2','i4'),('P_random','f8'),('P_binary','f8'),('P_posterior','f8')]
prob_out = np.array([], dtype=dtype)


for i in np.arange(length):
    
    if i%1000 == 0: print i
    # print i
        
    # Get ids of all stars within 1 degree
    i_star2 = np.arange(length - i - 1) + i + 1
    theta = get_ang_sep(t['ra'][i], t['dec'][i], t['ra'][i_star2], t['dec'][i_star2])
    ids_good = i_star2[np.where(theta < 1.0)[0]]
    
    
#     print np.where(theta < 1.0)[0]
#     print i_star2[np.where(theta < 1.0)[0]][0], ids_good[0]
#     print t['ra'][i], t['dec'][i], t['ra'][ids_good[0]], t['dec'][ids_good[0]], theta[ids_good[0]-i-1]
#     If a repeated entry, move on to the next one
#     if np.any(t['ID'][i] == t['ID'][ids_good]): continue
    

    

#     print i, t['ID'][i], len(ids_good), np.min(theta)*3600.0, t['ra'][i], t['dec'][i], t['ra'][np.argmin(theta)+i+1], t['dec'][np.argmin(theta)+i+1]
#     tmp_i = ids_good[np.argmin(theta[ids_good])] 
#     print i, tmp_i, t['ID'][tmp_i], t['ra'][tmp_i], t['dec'][tmp_i]
    
    
    # Move on if no matches within 1 degree
    if len(ids_good) == 0: continue
    
    delta_mu_ra_err = np.sqrt(t['mu_ra_err'][i]**2 + t['mu_ra_err'][ids_good]**2)
    delta_mu_dec_err = np.sqrt(t['mu_dec_err'][i]**2 + t['mu_dec_err'][ids_good]**2)

    
    delta_mu_ra_sample = multivariate_normal(mean=(t['mu_ra'][i] - t['mu_ra'][ids_good]), \
                                             cov=np.diag(delta_mu_ra_err), \
                                             size=size_integrate)
    delta_mu_dec_sample = multivariate_normal(mean=(t['mu_dec'][i] - t['mu_dec'][ids_good]), \
                                              cov=np.diag(delta_mu_dec_err), \
                                              size=size_integrate)
    delta_mu_sample = np.sqrt(delta_mu_ra_sample**2 + delta_mu_dec_sample**2)
    
    
    prob_tmp = P_binary.get_P_binary(np.repeat(theta[ids_good-i-1], size_integrate) * 3600.0, np.ravel(delta_mu_sample.T))
    prob_binary = 1.0/size_integrate * np.sum(prob_tmp.reshape((len(ids_good), size_integrate)), axis=1)    
    
    ids_good_binary = np.where(prob_binary > 0.0)[0]
    if len(ids_good_binary) == 0: continue
    ids_good_binary_all = ids_good[ids_good_binary]
    
    # for j in ids_good_binary_all:
    for k in np.arange(len(ids_good_binary_all)):
        j = ids_good_binary_all[k]

    
        # Star arrays
        star1 = t['ra'][i], t['dec'][i], t['mu_ra'][i], t['mu_dec'][i], t['mu_ra_err'][i], t['mu_dec_err'][i]
        star2 = t['ra'][j], t['dec'][j], t['mu_ra'][j], t['mu_dec'][j], t['mu_ra_err'][j], t['mu_dec_err'][j]
        theta_match = get_ang_sep(t['ra'][i], t['dec'][i], t['ra'][j], t['dec'][j])

        
        # Proper motion uncertainties
        delta_mu_ra_err = np.sqrt(t['mu_ra_err'][i]**2 + t['mu_ra_err'][j]**2)
        delta_mu_dec_err = np.sqrt(t['mu_dec_err'][i]**2 + t['mu_dec_err'][j]**2)

        
        # Recalculate binary probabilities
        delta_mu_ra_sample = normal(loc=(t['mu_ra'][i] - t['mu_ra'][j]), \
                                                 scale=delta_mu_ra_err, \
                                                 size=size_integrate_full)
        delta_mu_dec_sample = normal(loc=(t['mu_dec'][i] - t['mu_dec'][j]), \
                                                  scale=delta_mu_dec_err, \
                                                  size=size_integrate_full)
        delta_mu_sample = np.sqrt(delta_mu_ra_sample**2 + delta_mu_dec_sample**2)


        prob_tmp = P_binary.get_P_binary(theta_match * 3600.0, delta_mu_sample)
        prob_binary = 1.0/size_integrate_full * np.sum(prob_tmp)    


        
        
        # Random Alignment densities
        pos_density = P_random.get_sigma_pos(t['ra'][i], t['dec'][i], catalog=t, method='kde')
        pm_density = P_random.get_sigma_mu(t['mu_ra'][i], t['mu_dec'][i], catalog=t, method='kde')


        # Calculate random alignment probabilities
        prob_random, prob_pos, prob_mu = P_random.get_P_random_alignment(star1[0], star1[1], star2[0], star2[1],
                                          star1[2], star1[3], star2[2], star2[3],
                                          delta_mu_ra_err=delta_mu_ra_err, delta_mu_dec_err=delta_mu_dec_err,
                                          pos_density=pos_density, pm_density=pm_density,
                                          catalog=t)
        
        
        # Save those pairs with posterior probabilities above 50% 
        prob_posterior = f_bin * prob_binary / (prob_random + f_bin * prob_binary)
        
#         print "Potential match. i:", i, "j:", j, "P(binary):", prob_binary[ids_good_binary[k]], "P(random):", prob_random, \
#                 "P(posterior):", prob_posterior
        print "Potential match. i:", i, "j:", j, "P(binary):", prob_binary, "P(random):", prob_random, \
                "P(posterior):", prob_posterior

        
        if prob_posterior > 0.5:
            prob_temp = np.zeros(1, dtype=dtype)
            prob_temp[0] = i, j, t['ID'][i], t['ID'][j], prob_random, prob_binary, prob_posterior
            prob_out = np.append(prob_out, prob_temp)
#             prob_out = np.append(prob_out, [i, j, t['ID'][i], t['ID'][j], prob_random, prob_binary[ids_good_binary[k]], prob_posterior])


print "Elapsed time:", time.time() - start, "seconds"


We are testing 81227 stars...
0


KeyboardInterrupt: 

N=10, t=50 

N=20, t=93


In [81]:
print prob_out


[(13, 14, 59800815, 59800815, 0.0, 0.010827243613101764, 1.0)]
